# Introduction
The TEDX Endocrine Disruptor Chemical list has ~1400 compounds that are 'potential' endocrine disruptors and about 10% of those are labeled as being used in Unconventional Oil/Gas operations.  Given that FracFocus actually lists a lot (though not all) chemicals used in a fracking operation, we should be able to compare the lists.  Both lists use CAS numbers so the we can use those to find the intersection of two lists.   

Load in the CAS Numbers and names from the TEDX list
---

The [TEDX website](https://endocrinedisruption.org/interactive-tools/tedx-list-of-potential-endocrine-disruptors/search-the-tedx-list#sname=&searchfor=any&sortby=chemname&action=search&searchcats=all&sortby=chemname) allows us to download the entire curated list.  The code below uses a trimmed version that makes it easier for computer reading.  


In [1]:
# Code preamble
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns


results_dir = './out/'       # for local use
#results_dir = '../results/' # for CodeOcean
data_dir = './sources/'      # for local use
#data_dir = '../data/'         # for CodeOcean

In [2]:
TEDXfn = data_dir+'TEDX_EDC_trimmed.xls'
tx = pd.read_excel(TEDXfn, encoding = "ISO-8859-1")
tx = tx.groupby('CAS_Num',as_index=False)['TEDX_Chem_Name'].first()
# print sample and length of the TEDX list
print(f'Number of chemicals in trimmed TEDX list: {len(tx)}')
tx.head()  # show just the first five chemicals in the list


Number of chemicals in trimmed TEDX list: 1354


,CAS_Num,TEDX_Chem_Name
0,100-01-6,p-nitroaniline
1,100-02-7,4-nitrophenol
2,100-09-4,4-methoxybenzoic acid
3,100-41-4,ethylbenzene
4,100-42-5,styrene


### Make a list of the CAS numbers from the TEDX list

In [3]:
tx_cas = list(tx.CAS_Num.unique())

## Load the filtered FracFocus data

In [4]:
# pull data from the results directory where it is stored as a .csv file

df = pd.read_csv(results_dir+'filtered_df.csv')
df.date = pd.to_datetime(df.date)  # put date back into datetime format
print(f'Total number of chemical records in filtered data set: {len(df)}')

Total number of chemical records in filtered data set: 3014015


### Keep only those records that are in the TEDX list

In [5]:
df = df[df.bgCAS.isin(tx_cas)]
print(f'Total number of chemical records with TEDX overlap: {len(df)}')
print(f'  -- number records that have calculated mass:      {len(df[df.bgMass>0])}')
print(f'  -- with a total unique chemicals: {len(df.bgCAS.unique())}')

Total number of chemical records with TEDX overlap: 544311
  -- number records that have calculated mass:      475409
  -- with a total unique chemicals: 115


In [6]:
df = pd.merge(df,tx,left_on='bgCAS',right_on='CAS_Num',how='left',validate='m:1')
df.drop('CAS_Num',axis=1,inplace =True)

### Compute the number of events and some simple statistics

In [7]:
gb1 = df.groupby(['UploadKey','bgCAS'],as_index=False)['CASNumber'].count()
gb2 = gb1.groupby('bgCAS',as_index=False)['UploadKey'].count()
gb3a = df.groupby('bgCAS',as_index=False)['bgMass'].mean()
gb3a.columns = ['bgCAS','mean_mass_lbs']
gb3b = df.groupby('bgCAS',as_index=False)['bgMass'].sum()
gb3b.columns = ['bgCAS','total_mass_lbs']
gb3c = df.groupby('bgCAS',as_index=False)['bgMass'].max()
gb3c.columns = ['bgCAS','max_mass_lbs']
gb4 = df.groupby('bgCAS',as_index=False)['bgIngredientName','TEDX_Chem_Name'].first()
mg = pd.merge(gb2,gb3a)
mg = pd.merge(mg,gb3b)
mg = pd.merge(mg,gb3c)
mg = pd.merge(mg,gb4).sort_values(by='UploadKey',ascending=False)

# save the list to the results directory
mg.to_csv(results_dir+'TEDX_Chem_in_FF.csv')
# mg.to_excel(results_dir+'TEDX_Chem_in_FF.xlsx')

### Sorted to show most frequent
See the CSV file for the full list.

In [8]:
mg.head(30)

,bgCAS,UploadKey,mean_mass_lbs,total_mass_lbs,max_mass_lbs,bgIngredientName,TEDX_Chem_Name
71,67-56-1,66203,3977.628139,4.189158e+08,2.771049e+06,methanol,methanol
14,107-21-1,41667,2902.347082,1.343932e+08,1.316196e+06,"1,2-ethanediol",ethylene glycol
66,64-17-5,29981,4224.847152,1.220432e+08,3.707897e+05,ethanol,ethanol
13,107-19-7,27576,126.999639,3.154671e+06,5.453396e+04,2-propyn-1-ol,propargyl alcohol
21,111-30-8,27312,3854.642151,9.956155e+07,9.902205e+05,pentanedial,glutaraldehyde
35,12125-02-9,25648,3035.674201,7.715470e+07,4.490093e+05,ammonium chloride ((nh4)cl),ammonium chloride
25,111-76-2,20384,2132.437441,4.898209e+07,4.302192e+05,"ethanol, 2-butoxy-",ethylene glycol monobutyl ether
76,7173-51-5,14131,1441.191628,1.956274e+07,2.477550e+05,"1-decanaminium, n-decyl-n,n-dimethyl-, chlorid...",didecyl dimethyl ammonium chloride
96,7758-19-2,14013,5239.430068,6.717997e+07,2.523339e+06,"chlorous acid, sodium salt (1:1)",sodium chlorite
110,9005-65-6,13253,1643.224571,1.863745e+07,1.181340e+05,"sorbitan, mono-(9z)-9-octadecenoate, poly(oxy-...",polysorbate 80


### Sorted to show chemicals with largest mean mass

In [9]:
mg.sort_values(by='mean_mass_lbs',ascending=False).head(30)

,bgCAS,UploadKey,mean_mass_lbs,total_mass_lbs,max_mass_lbs,bgIngredientName,TEDX_Chem_Name
82,7447-40-7,6015,71998.203690,3.704308e+08,2.585980e+07,potassium chloride (kcl),potassium chloride
104,8002-05-9,3,60223.837508,6.022384e+04,6.022384e+04,petroleum,petroleum distillate naphtha
89,7631-99-4,775,16228.031710,1.415084e+07,4.952183e+05,nitric acid sodium salt (1:1),sodium nitrate
102,7789-38-0,642,12183.852028,6.749854e+06,1.510422e+06,"bromic acid, sodium salt (1:1)",sodium bromate
46,13463-67-7,4132,10419.385206,4.628291e+07,1.154219e+06,titanium oxide (tio2),titanium dioxide nanoparticles
5,10049-04-4,4897,9063.889135,3.962732e+07,5.145341e+06,chlorine oxide (clo2),chlorine dioxide
77,7429-90-5,5,8324.926679,4.162463e+04,9.421821e+03,aluminum,aluminum
45,1344-09-8,3,8014.720972,2.404416e+04,9.045530e+03,"silicic acid, sodium salt",sodium silicate
70,65-85-0,270,6937.177972,1.789792e+06,2.662308e+05,benzoic acid,benzoic acid
57,51200-87-4,2150,6276.734446,1.316231e+07,4.335614e+05,"oxazolidine, 4,4-dimethyl-","4,4’-dimethyl-1,3-oxazoline"
